In [5]:
import pandas as pd

movieNames = pd.read_table("ml-100k/u.item",names = ["movieID","title","date","_","URL"],
                           usecols = ["movieID","title","date","_","URL"],
                    sep ="|",index_col = "movieID", encoding = "cp1252")
movieNames.drop(columns=["date","_"],inplace=True)

In [6]:
movieNames.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1682 entries, 1 to 1682
Data columns (total 2 columns):
title    1682 non-null object
URL      1679 non-null object
dtypes: object(2)
memory usage: 39.4+ KB


In [7]:
movieNames["year"] = movieNames["title"].str.slice(-5,-1)
movieNames["title"] = movieNames["title"].str.slice(0,-6)

In [8]:
movieNames.head()

,title,URL,year
movieID,,,
1,Toy Story,http://us.imdb.com/M/title-exact?Toy%20Story%2...,1995
2,GoldenEye,http://us.imdb.com/M/title-exact?GoldenEye%20(...,1995
3,Four Rooms,http://us.imdb.com/M/title-exact?Four%20Rooms%...,1995
4,Get Shorty,http://us.imdb.com/M/title-exact?Get%20Shorty%...,1995
5,Copycat,http://us.imdb.com/M/title-exact?Copycat%20(1995),1995


In [9]:
import sqlite3
conn = sqlite3.connect('movieNames.db')

In [10]:
conn.execute('''CREATE TABLE IF NOT EXISTS movies
             (movieID INTEGER PRIMARY KEY, title TEXT, URL TEXT,year INTEGER)''')

In [11]:
# Create table
conn.execute('''CREATE VIRTUAL TABLE IF NOT EXISTS searchableNames 
USING FTS5(title, movieID, year);''')

In [12]:
conn.commit()

In [13]:
for i in movieNames.itertuples(name=None):
    #print(i)
    conn.execute('''INSERT INTO movies(movieID,title,URL,year)
VALUES(?,?,?,?);''',i)
    conn.execute('''INSERT INTO searchableNames(title,movieID,year)
VALUES(?,?,?);''',(i[1],i[0],i[3]))
conn.commit()
conn.close()

In [7]:
import sqlite3

def getMovieFromFreeText(conn,text):
    c = conn.cursor()
    c.execute('''SELECT * 
FROM searchablenames
WHERE searchablenames.title MATCH ? ORDER BY rank;''',(text,))
    return c.fetchall()

conn = sqlite3.connect('movieNames.db')
print(getMovieFromFreeText(conn,"godfather"))
conn.close()

[('Godfather, The ', 127, '1972'), ('Godfather: Part II, The ', 187, '1974')]


In [2]:
def getMovieFromID(conn,movieID):
    c = conn.cursor()
    c.execute('''SELECT * 
FROM movies
WHERE movies.movieID=?;''',(movieID,))
    return c.fetchone()

In [3]:
import sqlite3

conn = sqlite3.connect('movieNames.db')
print(getMovieFromID(conn,1675))
conn.close()

(1675, 'Sunchaser, The ', 'http://us.imdb.com/M/title-exact?Sunchaser,%20The%20(1996)', 1996)


In [4]:
def searchRecommendation(conn,movieID,scoreThreshold,coOccurenceThreshold):
    c = conn.cursor()
    c.execute('''SELECT * 
FROM moviePairs as mp
WHERE (mp.movie1=:movieID OR mp.movie2=:movieID) AND numPairs>:coT AND score>:sT
ORDER BY score DESC LIMIT 10;''',
              {"movieID":movieID, "coT":coOccurenceThreshold, "sT":scoreThreshold })
    return c.fetchall()

In [9]:
import sqlite3

movieID = 182
scoreThreshold = 0.97
coOccurenceThreshold = 50

conn = sqlite3.connect('movieNames.db')

rec = searchRecommendation(conn,movieID,scoreThreshold,coOccurenceThreshold )

for row in rec:
    if movieID == row[0]:
        movieRec = getMovieFromID(conn,row[1])
    else:
        movieRec = getMovieFromID(conn,row[0])
    recDic = {"movieId":movieRec[0],
              "title":movieRec[1],
              "URL":movieRec[2],
              "year":movieRec[3],
              "score":row[3],
              "pairs":row[2]
             }
    print(recDic)

conn.close()

{'movieId': 693, 'title': 'Casino ', 'URL': 'http://us.imdb.com/M/title-exact?Casino%20(1995)', 'year': 1995, 'score': 0.9852812012394672, 'pairs': 79}
{'movieId': 504, 'title': 'Bonnie and Clyde ', 'URL': 'http://us.imdb.com/M/title-exact?Bonnie%20and%20Clyde%20(1967)', 'year': 1967, 'score': 0.9799131760011474, 'pairs': 73}
{'movieId': 285, 'title': 'Secrets & Lies ', 'URL': 'http://us.imdb.com/M/title-exact?Secrets%20&%20Lies%20(1996)', 'year': 1996, 'score': 0.978287431271312, 'pairs': 53}
{'movieId': 177, 'title': 'Good, The Bad and The Ugly, The ', 'URL': 'http://us.imdb.com/M/title-exact?Buono,%20il%20brutto,%20il%20cattivo,%20Il%20(1966)', 'year': 1966, 'score': 0.9767310633242275, 'pairs': 88}
{'movieId': 127, 'title': 'Godfather, The ', 'URL': 'http://us.imdb.com/M/title-exact?Godfather,%20The%20(1972)', 'year': 1972, 'score': 0.9763210238904455, 'pairs': 169}
{'movieId': 187, 'title': 'Godfather: Part II, The ', 'URL': 'http://us.imdb.com/M/title-exact?Godfather:%20Part%20II